In [83]:
import pandas as pd

## Get MCBS

In [84]:
mcbs = pd.read_csv("../data/csv/nicoalco.csv")

In [85]:
len(mcbs)

12297

In [86]:
df = pd.read_csv("../data/csv/demo.csv") # demographics file

/n/helmod/apps/centos7/Core/Anaconda3/2020.11-jupyterood-fasrc01/x/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (24,25,26,27,28,29,30,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [87]:
len(df)

14068

## Get the MCBS-MBSF cross-walk

In [88]:
xwalk = pd.read_csv("../data/mcbs-xwalk/mcbs_xwalk_file_res000017155_req010441_2018.dat", names=["xwalk"])

In [89]:
# get MCBS ids from xwalk
xwalk["mcbs"]=xwalk["xwalk"].str[:8]
# get MBSF ids
xwalk["mbsf"]=xwalk["xwalk"].str[8:]
xwalk["mcbs"]=xwalk["mcbs"].astype(int)

In [90]:
xwalk=xwalk.drop(columns=['xwalk'])

## Merge MCBS and MBSF cross-walk

In [91]:
mcbs_x = mcbs.merge(xwalk, left_on='BASEID', right_on='mcbs', how='inner')

In [92]:
df_x = df.merge(xwalk, left_on='BASEID', right_on='mcbs', how='inner')

In [93]:
len(mcbs_x)

12297

In [94]:
len(df_x)

14068

## Get MBSF (enrollment)

All MBSF columns:

cols = ["zip","year","qid","dodflag","bene_dod","sex","race","age","hmo_mo","hmoind","statecode","latitude","longitude",
        "dual","death","dead","entry_age","entry_year","entry_age_break","followup_year","followup_year_plus_one","pm25_ensemble",
        "pm25_no_interp","pm25_nn","ozone","ozone_no_interp","zcta","poverty","popdensity","medianhousevalue","pct_blk",
        "medhouseholdincome","pct_owner_occ","hispanic","education","population","zcta_no_interp","poverty_no_interp",
        "popdensity_no_interp","medianhousevalue_no_interp","pct_blk_no_interp","medhouseholdincome_no_interp",
        "pct_owner_occ_no_interp","hispanic_no_interp","education_no_interp","population_no_interp","smoke_rate","mean_bmi",
        "smoke_rate_no_interp","mean_bmi_no_interp","amb_visit_pct","a1c_exm_pct","amb_visit_pct_no_interp",
        "a1c_exm_pct_no_interp","tmmx","rmax","pr","cluster_cat","fips_no_interp","fips","summer_tmmx",
        "summer_rmax","winter_tmmx","winter_rmax"]

In [95]:
cols = ["zip","year","qid","dodflag","bene_dod","sex","race","age","hmo_mo","hmoind","statecode",
        "dual","death","dead","entry_age","entry_year","entry_age_break","followup_year"]

In [120]:
mbsf = pd.read_csv(
    "../data/denom_by_year_csv/confounder_exposure_merged_nodups_health_2015.csv", usecols=cols)

/n/helmod/apps/centos7/Core/Anaconda3/2020.11-jupyterood-fasrc01/x/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Merge onto MCBS nicoalco and demo:

In [121]:
mcbs_x_mbsf = mcbs_x.merge(
    mbsf, left_on='mbsf', right_on='qid', how='left', suffixes=(None, "_2015"))
# An inner merge leads to: len(mcbs_x_mbsf) =  10042

In [122]:
df_x_mbsf = df_x.merge(
    mbsf, left_on='mbsf', right_on='qid', how='left', suffixes=(None, "_2015"))
# An inner merge leads to: len(df_x_mbsf) =  11585

Get MBSF 2016 and merge onto the merged MCBS x MBSF 2015

In [123]:
mbsf = pd.read_csv(
    "../data/denom_by_year_csv/confounder_exposure_merged_nodups_health_2016.csv", usecols=cols)

In [124]:
mcbs_x_mbsf = mcbs_x_mbsf.merge(
    mbsf, left_on='mbsf', right_on='qid', how='left', suffixes=(None, "_2016"))

In [125]:
df_x_mbsf = df_x_mbsf.merge(
    mbsf, left_on='mbsf', right_on='qid', how='left', suffixes=(None, "_2016"))

In [126]:
len(mcbs_x_mbsf)

12297

In [127]:
len(df_x_mbsf)

14068

## Cleanup and save

In [128]:
mcbs_x_mbsf = mcbs_x_mbsf.drop(columns=['mcbs', 'mbsf','qid_2016'])
df_x_mbsf = df_x_mbsf.drop(columns=['mcbs', 'mbsf','qid_2016'])

Apply "age" >= 65 in 2015 cut

In [129]:
# mcbs_x_mbsf["age"]=pd.to_numeric(mcbs_x_mbsf["age"])
df_x_mbsf = df_x_mbsf[df_x_mbsf["H_AGE"] >= 65]

Select for the 2015 panel.

In [130]:
df_x_mbsf= df_x_mbsf[df_x_mbsf["PANEL"]==2015]

Select the same cohort in nicoalco:

In [131]:
mcbs_x_mbsf = mcbs_x_mbsf[
    mcbs_x_mbsf["BASEID"].isin(df_x_mbsf["BASEID"])]

In [132]:
len(mcbs_x_mbsf)

3392

In [133]:
len(df_x_mbsf)

3612

In [134]:
mcbs_x_mbsf["dead"].sum() # dead in 2015

29

In [135]:
mcbs_x_mbsf["dead_2016"].sum() # dead in 2016

148

In [136]:
mcbs_x_mbsf.to_csv("../data/mcbs_nicoalco_x_mbsf_15-16.csv", index=False)
df_x_mbsf.to_csv("../data/mcbs_demo_x_mbsf_15-16.csv", index=False)